# tensorflow版本代码 用来做验证

In [21]:
#
# 

# from keras.utils import to_categorical#transform one-hot vector

In [9]:
import torch.nn as nn
import tensorflow as tf
import pickle
import torch.nn.functional as F
import torch
from torch.autograd import Variable
import numpy as np
from torch.nn import CrossEntropyLoss

In [35]:
initial = tf.constant(0.1,shape=[100])
initial.shape

TensorShape([100])

In [44]:
output1=tf.constant(2.0,shape=[32,1,1,100])

In [54]:
x_in = np.array([[
  [[2], [1], [2], [0], [1]],
  [[1], [3], [2], [2], [3]],
  [[1], [1], [3], [3], [0]],
  [[2], [2], [0], [1], [1]],
  [[0], [0], [3], [1], [2]], ]])
kernel_in = np.array([
 [ [[2, 0.1,2,2,2]], [[3, 0.2,2,2,2]] ],
 [ [[0, 0.3,2,2,2]],[[1, 0.4,2,2,2]] ], ])
input1 = tf.constant(1,shape=[32,40,300,1], dtype=tf.float32)
kernel = tf.constant(0.1,shape=[3,300,1,100], dtype=tf.float32)
print(x.shape,kernel.shape)
out1=tf.nn.conv2d(input1, kernel, strides=[1, 1, 1, 1], padding='VALID')+initial
print(out1.shape)
h_conv1=tf.nn.relu(out1)
h_conv1.shape

(32, 40, 300, 1) (3, 300, 1, 100)
(32, 38, 1, 100)


TensorShape([32, 38, 1, 100])

In [40]:
h_poo11 = tf.nn.max_pool(h_conv1,ksize=[1,input1.shape[1]-3+1,1,1],strides=[1,1,1,1],padding='VALID')
h_poo11.shape

TensorShape([32, 1, 1, 100])

In [47]:
output1 = tf.concat([output1,h_poo11],axis=3) #把卷积核得出的特征向量连接起来

In [48]:
output1.shape

TensorShape([32, 1, 1, 300])

In [49]:
input2=tf.reshape(output1,[-1,8,300,1])
input2.shape

TensorShape([4, 8, 300, 1])

In [40]:
#测试NPCNN_hdj的代码数据加载部分
print("loading data for training")
test_num = 58
X = pickle.load(open('/home/hdj/NPCNN_hdj/parameters.in','rb'))
train_report,train_code,train_labels,W = X[0],X[1],X[2],X[3]
embedding = nn.Embedding.from_pretrained(torch.tensor(W))#从已有的weight导入embedding
train_code = embedding(torch.tensor(train_code))
train_report =embedding(torch.tensor(train_report))
print(train_code.shape,train_report.shape)
#被废弃的效率极低的代码
# train_report = np.array(train_report)
# train_code = np.array(train_code)
# train_labels = np.array(train_labels)
# # train_labels = to_categorical(train_labels)
# W = np.array(W)
# print(np.array(train_report).shape, np.array(train_code).shape, np.array(train_labels).shape, np.array(W).shape)
print("finish reading")

loading data for training
(2108, 308) (2108, 100, 40) (2108,) (4206, 300)
finish reading


In [46]:
embedding = nn.Embedding.from_pretrained(torch.tensor(W))
train_code = embedding(torch.tensor(train_code))
train_report =embedding(torch.tensor(train_report))
print(train_code.shape,train_report.shape)

torch.Size([2108, 100, 40, 300]) torch.Size([2108, 308, 300])


In [22]:
#这一步操作及其慢，需要优化
W[[[[0,1]]]].shape

/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


(1, 2, 300)

In [41]:
#测试torch的embedding使用
embedding = nn.Embedding(4206, 300)
# a batch of 2 samples of 4 indices each
input = torch.tensor(train_report)
print(input.shape)


torch.Size([2108, 308])


In [43]:
codeTensor = torch.tensor(train_code)
output=embedding(codeTensor)
output.shape

torch.Size([2108, 100, 40, 300])

In [42]:
output=embedding(input)
output.shape

torch.Size([2108, 308, 300])

In [16]:
train_labels

array([1, 0, 1, ..., 0, 1, 1])

In [ ]:
# train_code[0][0:10]

In [2]:
# train_code[0][0:10]

In [3]:
# train_report[1][2]

In [4]:
# train_report[1]


In [ ]:
train_code = W[train_code]
train_report = W[train_report]
print(np.array(train_code).shape,np.array(train_report).shape)

In [ ]:
loading data for training
(3292, 208) (3292, 50, 20) (3292,) (3578, 300)
finish reading
(3292, 50, 20, 300) (3292, 208, 300)

# 这里使用CNN来进行code和report的编码 之后和codeBert的信息进行融合看能否得到提升

# 模型定义 code是两次卷积 report是一次卷积

In [20]:
import torch
import pickle
from tqdm import tqdm, trange
import torch.nn as nn
# from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, AdamW
import transformers
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import random
from sklearn import metrics
import torch.nn as nn
import argparse
import torch.distributed as dist
import torch.utils.data.distributed
import os
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from utils import (compute_metrics, convert_examples_to_features,
                        output_modes, processors)
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
# from transformers import (WEIGHTS_NAME, get_linear_schedule_with_warmup, AdamW,
#                           RobertaConfig,
#                           RobertaForSequenceClassification,
#                           RobertaTokenizer)
from CodeBertModel import TextCNNClassifer
import os
# for i in range(58):
#     os.system('python3 run_classifier.py --model_type roberta --model_name_or_path microsoft/codebert-base --task_name codesearch --do_predict --output_dir ../data/codesearch/test/ --data_dir ../data/codesearch/test/ --max_seq_length 512 --per_gpu_train_batch_size 32 --per_gpu_eval_batch_size 32 --learning_rate 1e-5 --num_train_epochs 8 --test_file aspectj_'+str(i)+'.txt  --pred_model_dir ./models/java/checkpoint-best/ --test_result_dir ./results/java/batch_result_'+str(i)+'.txt')
from more_itertools import chunked
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')
def calculate_same_value(labels_sorted, test_p_sorted, start_pos):
    i = start_pos
    num_same = 0
    num_p = 0
    while test_p_sorted[start_pos] == test_p_sorted[i]:
        num_same = num_same + 1
        if labels_sorted[i] ==1 : num_p = num_p + 1
        i = i + 1
        if i == len(labels_sorted ): break
    return num_p, num_same
def eval_mrr(test_p, labels):#在第二维相似度得分，真实标签
    test_p_sorted = test_p
    test_p_index = sorted(range(len(test_p_sorted)), key=lambda k: test_p_sorted[k], reverse=True)  # 降序排序
    test_p_sorted = sorted(test_p, reverse=True)

    labels_sorted = []
    for index in test_p_index:
        labels_sorted.append(labels[index])

    top_num = 10
    top10rank = 0
    for i in range(top_num):
        if (labels_sorted[i] == 1):
            '''
            num_p, num_s = calculate_same_value(labels_sorted, test_p_sorted, i)
            num_r = top_num - i
            if num_p > (num_s-num_r):
                top10rank = 1
                break
            v1 = perm(num_s-num_r, num_p)*perm(num_s-num_p,num_s-num_p)
            v2 = perm(num_s,num_s)
            top10rank = 1-(float)((float)(v1)/(float)(v2))
            if top10rank > 1: top10rank=1
            if top10rank!=top10rank: top10rank=1
            break

    return top10rank

    '''
            top10rank = 1
            break
    num_p, num_s = calculate_same_value(labels_sorted, test_p_sorted, 10)
    if (num_p >= 1): top10rank = 1  # 统计在第十位并列排名相同的文件中，是否含有相关文件

    MRRrank = 0.0
    for i in range(len(labels_sorted)):
        if (labels_sorted[i] == 1):
            MRRrank = MRRrank + float(1 / (i + 1))
            break

    MAPrank = 0.0
    pos_num = 0
    for i in range(len(labels_sorted)):
        if (labels_sorted[i] == 1):
            pos_num = pos_num + 1
            MAPrank = MAPrank + float(pos_num / (i + 1))
    if pos_num==0:
        print('出现不存在pos的例子')
        pos_num=1
    MAPrank = float(MAPrank / pos_num)
#     MRRrank = float(MRRrank / pos_num)
    
    return top10rank, MRRrank, MAPrank

def simple_accuracy(preds, labels):
#     print(type(preds),type(labels))
    return (preds == labels).mean()


def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds)
    return {
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
    }


def compute_metrics(preds, labels):
    assert len(preds) == len(labels)

    return acc_and_f1(preds, labels)
#设置种子，为了结果复现
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
def minibatches(inputs_x1 = None,inputs_x2 = None,targets = None,batch_size = None,shuffle = False):
    if shuffle:#判断是否混合
        indices = np.arange(len(inputs_x1))
        np.random.shuffle(indices)
    for start_idx in range(0,len(inputs_x1)-batch_size+1,batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx+batch_size]
        else:
            excerpt = slice(start_idx,start_idx+batch_size)
        yield inputs_x1[excerpt],inputs_x2[excerpt],targets[excerpt]

def get_testsets(test_code,test_report,test_label):
    code_x = test_code[0]
    for i in range(300-1):
        code_x = np.vstack((code_x,test_code[i+1]))
    report_x = test_report
    y = test_label
    return code_x,report_x,y
def calculate_same_value(labels_sorted, test_p_sorted, start_pos):
    i = start_pos
    num_same = 0
    num_p = 0
    while test_p_sorted[start_pos] == test_p_sorted[i]:
        num_same = num_same + 1
        if labels_sorted[i] ==1 : num_p = num_p + 1
        i = i + 1
        if i == len(labels_sorted ): break
    return num_p, num_same

def eval_y (test_p, labels ):
    test_p_sorted = test_p
    test_p_index = sorted(range(len(test_p_sorted)) ,key = lambda k: test_p_sorted[k], reverse =True )#降序排序
    test_p_sorted = sorted(test_p, reverse = True)

    labels_sorted = []
    for index in test_p_index:
        labels_sorted.append(labels[index])

    top_num = 10
    top10rank = 0
    for i in range(top_num):
        if(labels_sorted[i] ==1 ):
            '''
            num_p, num_s = calculate_same_value(labels_sorted, test_p_sorted, i)
            num_r = top_num - i
            if num_p > (num_s-num_r):
                top10rank = 1
                break
            v1 = perm(num_s-num_r, num_p)*perm(num_s-num_p,num_s-num_p)
            v2 = perm(num_s,num_s)
            top10rank = 1-(float)((float)(v1)/(float)(v2))
            if top10rank > 1: top10rank=1
            if top10rank!=top10rank: top10rank=1
            break

    return top10rank

    '''
            top10rank = 1
            break
    num_p, num_s = calculate_same_value(labels_sorted,test_p_sorted,10)
    if(num_p >=1 ): top10rank = 1 #统计在第十位并列排名相同的文件中，是否含有相关文件

    MRRrank = 0.0
    for i in range(len(labels_sorted)):
        if( labels_sorted[i] == 1 ):
            MRRrank = MRRrank +  float(1/(i+1))
            break

    MAPrank = 0.0
    pos_num = 0
    for i in range(len(labels_sorted)):
        if( labels_sorted[i] == 1):
            pos_num = pos_num + 1
            MAPrank = MAPrank + float(pos_num/(i+1))
    try:
        MAPrank =  float(MAPrank/pos_num)
    except:
        print('出现pos_num为0的情况',pos_num)
        return 1,1,1
#     MRRrank = float(MRRrank/pos_num)
   
    return top10rank, MRRrank, MAPrank


In [68]:
#测试torch代码所用
# #输入[batch_size,in_channels,height,weight]
# #输出 [batch_size,out_channles,H_out,W_out]
# inputCode = torch.randn(32, 1, 40, 300)
# conv2 = nn.Conv2d(in_channels=1, out_channels=100, kernel_size=(3, 300), stride=(1, 1), bias=True)
# out=conv2(inputCode)
# print(out.shape)
# # out

# import torch.nn.functional as F
# outRelu=F.relu(out)
# print(outRelu.shape)
# # pool of square window of size=3, stride=2
# # m = nn.MaxPool2d(3, stride=2)
# # pool of non-square window
# m = nn.MaxPool2d(kernel_size=(38, 1), stride=(1, 1))
# input = torch.randn(32, 100, 38, 1)
# output = m(input)
# print(output.shape)
# listOut=[output,output]
# out = torch.cat(listOut, dim=1)
# print(out.shape)
# out = out.view(-1, 1, 8,200)
# print(out.shape)

torch.Size([32, 100, 38, 1])


# 模型配置和定义部分

In [2]:
class TCNNConfig(object):
    def __init__(self):
        self.in_channels=1
        self.out_channels=100
        self.embedding_size=300
        self.vocab_size=100
        self.wordNums=40
        self.codeLineNums=100
        self.reportLineNums=308
        self.window_sizes=[3,4,5]
        self.num_class=2
config=TCNNConfig()
print(config.in_channels,config.out_channels)
#torch 
'''
in_channels(int) – 输入信号的通道
out_channels(int) – 卷积产生的通道
kerner_size(int or tuple) - 卷积核的尺寸
stride(int or tuple, optional) - 卷积步长
padding(int or tuple, optional) - 输入的每一条边补充0的层数
dilation(int or tuple, optional) – 卷积核元素之间的间距
groups(int, optional) – 从输入通道到输出通道的阻塞连接数
bias(bool, optional) - 如果bias=True，添加偏置
'''
class CodeTextCNN(nn.Module):
    def __init__(self, config):
        super(CodeTextCNN, self).__init__()
        self.config=config
#         self.embedding = nn.Embedding(num_embeddings=config.vocab_size, embedding_dim=config.embedding_size)
        #(32, 1, 40, 300)
        self.convs = nn.ModuleList([
            nn.Sequential(
                          nn.Conv2d(in_channels=config.in_channels, out_channels=config.out_channels, kernel_size=(h,config.embedding_size ), stride=(1, 1), bias=True),
                          #                              nn.BatchNorm1d(num_features=config.feature_size),
                          nn.ReLU(),
                          nn.MaxPool2d(kernel_size=(config.wordNums-h+1, 1), stride=(1, 1))
            )
#                           nn.MaxPool1d(kernel_size=config.max_text_len - h + 1))
            for h in config.window_sizes
        ])
        #(32, 1, 100,300)
        self.convs_lines = nn.ModuleList([
            nn.Sequential(nn.Conv2d(in_channels=config.in_channels, out_channels=config.out_channels, kernel_size=(h, config.out_channels*len(config.window_sizes)), stride=(1, 1), bias=True),
                          #                              nn.BatchNorm1d(num_features=config.feature_size),
                          nn.ReLU(),
                          nn.MaxPool2d(kernel_size=(config.codeLineNums-h+1, 1), stride=(1, 1))
                         )
            for h in config.window_sizes
        ])
    
        self.report_convs = nn.ModuleList([
            nn.Sequential(nn.Conv2d(in_channels=config.in_channels, out_channels=config.out_channels, kernel_size=(h, config.embedding_size), stride=(1, 1), bias=True),
                          #                              nn.BatchNorm1d(num_features=config.feature_size),
                          nn.ReLU(),
                          nn.MaxPool2d(kernel_size=(config.reportLineNums-h+1, 1), stride=(1, 1))
                         )
            for h in config.window_sizes
        ])
#         self.fc0=nn.Linear(in_features=config.out_channels*len(config.window_sizes)*2,out_features=config.out_channels*len(config.window_sizes)*2,bias=True)
        self.fc1 = nn.Linear(in_features=config.out_channels*len(config.window_sizes) * 2,out_features=config.num_class,bias=True)
#         self.fc0=nn.Linear(in_features=config.embedding_size*2,out_features=config.embedding_size*2,bias=True)
#         self.fc1 = nn.Linear(in_features=config.embedding_size * 2,out_features=config.num_class,bias=True)
#         self.drop=nn.Dropout(0.1)
        self.dropout = nn.Dropout(0.5)
        self.classifier=nn.Linear(in_features=768*2,out_features=config.num_class,bias=True)
        # if os.path.exists(config.embedding_path) and config.is_training and config.is_pretrain:
        # print("Loading pretrain embedding...")
        # self.embedding.weight.data.copy_(torch.from_numpy(np.load(config.embedding_path)))
#         self.transformer=RobertaModel.from_pretrained("microsoft/codebert-base")
    def forward(self, x_code,y_report,report_ids,report_mask,code_ids,code_mask):
        #这是codeBert的编码
#         h_report = self.transformer(input_ids=report_ids, attention_mask=report_mask)
#         out_report=h_report[1]
#         h_code = self.transformer(input_ids=code_ids, attention_mask=code_mask)
#         out_code=h_code[1]
# #         abs_dist = torch.abs(torch.add(out_report, -out_code))
#         abs_dist=torch.cat((out_report,out_code),1)
#         codeBertOut=self.classifier(abs_dist)
#         return codeBertOut
        #这是codeBert的编码
        
        # embed_x = self.embedding(x)self.out_channels
        embed_x = x_code.view(-1,1,self.config.wordNums,config.embedding_size)#[32*100,40,300]-->[32*100,1,40,300]
#         embed_y=y_report.view(-1,1,self.config.reportLineNums,config.out_channels*len(config.window_sizes))#[32,150,300]-->[32,1,150,300]
#         embed_x = x_code.view(-1,1,self.config.wordNums,config.embedding_size)#[32*100,40,300]-->[32*100,1,40,300]
        embed_y=y_report.view(-1,1,self.config.reportLineNums,config.embedding_size)#[32,150,300]-->[32,1,150,300]
#         print('embed_x ',embed_x.shape,' embed_y ',embed_y.shape)
        out = [conv(embed_x) for conv in self.convs]  #[32*100,1,40,300]--> [32*100,100,38,1]--->pool [32*100,100,1,1]
        out = torch.cat(out, dim=1)
#         print('code第一次拼接的shape',out.shape)#[32*100, 300, 1, 1]
        out = out.view(-1, 1, config.codeLineNums,config.out_channels*len(config.window_sizes))#[32,1,100,300]
#         out = out.view(-1, 1, config.codeLineNums,config.embedding_size)#[32,1,100,300]
#         print('code第一次卷积',out.shape)

        # 开始在行间做二次卷积
        out2 = [conv(out) for conv in self.convs_lines]#[32,1,100,300]-->[32,100,98,1]-->pool[32,100,1,1]
#         for o in out2:
#             print('o2', o.size())  # ([32, 100, 1, 1])
        outMerge = torch.cat(out2, dim=1)  # [32,300,1,1]
#         print('code二次卷积size：',outMerge.size())  # [32,300,1,1]
        
        report_out = [conv(embed_y) for conv in self.report_convs]  # [32,1,150,300]-->[32,100,148,1]-->pool[32,100,1,1]
        report_outMerge = torch.cat(report_out, dim=1)#[32,300,1,1]
#         print('report 卷积size:',report_outMerge.size())
        report_outMerge = report_outMerge.view(-1, report_outMerge.size(1))#
        
        outMerge = outMerge.view(-1, outMerge.size(1))
#         print('code size report size : ',outMerge.size(),report_outMerge.size())#torch.Size([32, 300]) torch.Size([32, 300])
        #合并code和report编码得到的向量
        merge=torch.cat([outMerge,report_outMerge],dim=1)
#         print('merge得到的size大小：',merge.shape)
#         merge=self.drop(merge)
#         output0=self.fc0(merge)
        merge = self.dropout(merge)
        output1=self.fc1(merge)
#         print('fc后的输出size',output1.shape)
        # if not self.use_element:
        #     out2 = F.dropout(input=out2, p=self.dropout_rate)
        #     out2 = self.fc(out2)
#         return out2,report_out
#         cos_simi=torch.mean(torch.cosine_similarity(outMerge, report_outMerge, dim=1))
        return output1
#         return outMerge,report_outMerge,output1
model=CodeTextCNN(config)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
_ = model.to(device)

1 100


In [30]:
input1 = torch.randn(8, 2)
input2 = torch.randn(8, 2)
cos_simi=torch.cosine_similarity(input1, input2, dim=1)
mean_cos_simi=torch.mean(cos_simi)
print(cos_simi.shape,mean_cos_simi,cos_simi)

torch.Size([8]) tensor(0.3583) tensor([ 9.5252e-04, -7.2193e-01,  2.1057e-01,  9.7696e-01,  8.7698e-01,
         9.3521e-01,  6.4361e-01, -5.6055e-02])


In [3]:
class args(object):
    """A single set of features of data."""

    def __init__(self):
        self.model_type = 'roberta'
        self.output_dir='/data/hdj/data/CodeBERT/codesearch/models/java'
        self.test_result_dir='/data/hdj/data/CodeBERT/codesearch/results/java/test_v1_0414.txt'
        self.start_epoch=0
        self.num_train_epochs=1
        self.model_type='roberta'
        self.config_name=''
        self.model_name_or_path=None
        self.task_name='codesearch'
        self.tokenizer_name=''
        self.model_name_or_path='microsoft/codebert-base'
        self.do_lower_case=True
        self.seed=42
        self.gradient_accumulation_steps=1
        self.weight_decay=0.0
        self.max_grad_norm=1.0
        self.learning_rate=1e-4
        self.adam_epsilon=1e-8
        self.warmup_steps=0
        self.max_steps=-1
        self.num_train_epochs=6
# class args(object):
#     """A single set of features of data."""

#     def __init__(self):
#         self.model_type = 'codesearch'
#         self.output_dir='/data/hdj/data/CodeBERT/codesearch/models/java'
#         self.test_result_dir='/data/hdj/data/CodeBERT/codesearch/results/java/test_v1_0414.txt'
#         self.start_epoch=0
#         self.num_train_epochs=1
#         self.model_type='roberta'
args=args()
#带权重的交叉熵
# weights=torch.tensor([0.2,0.8]).cuda()
# loss_fun=CrossEntropyLoss(weight=weights)
#更换损失函数
# lossRankFunction = nn.MarginRankingLoss()
#不带权重的交叉熵
loss_fun=CrossEntropyLoss()
#设置种子 复现结果
set_seed(args)

In [4]:
from transformers import (WEIGHTS_NAME, get_linear_schedule_with_warmup, AdamW)
early_stop=3
eval_loss=0.0


if  True:
    #Google: 使用的优化器和迭代器
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    #Google
else:
#     hdj: textCNNClassifier 使用的优化器和计划器
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-04)#, betas=(0.9, 0.999)
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
#     hdj


In [15]:
# print('report shape :',train_x2.shape,' code shape:',train_x1.shape,' train_labels shape:',torch.tensor(train_labels).shape,' embedding shape:',embedding.num_embeddings)
# #15:40--16:12

In [5]:
#需要为数据的code生成文本形式
def code_merge(raw_code_list):
    code_list=[]
    for raw_code in raw_code_list:
        code_list.append(raw_code['text'])
    code_text=' '.join(code_list)
    return code_text

def data_tensor_generate(data,project,tokenizer,maxLength=512,ttype='train'):
    '''
        输入是dataframe 包含bug_id	cnn_report_idx	code_idx	label	path	raw_report	raw_code
    '''
    #将raw_code里dict形式合并生成code_text
    data['raw_code_text']=data['raw_code'].apply(code_merge)
    #生成完成
    all_labels=list(data['label'])
    all_cnn_report_idx=list(data['cnn_report_idx'])
    all_code_idx=list(data['code_idx'])
    raw_report=list(data['raw_report'])
    raw_code=list(data['raw_code_text'])
#     reportInputs=tokenizer.batch_encode_plus(raw_report,add_special_tokens=True,max_length=maxLength,pad_to_max_length=True,return_token_type_ids=True,truncation=True)
#     codeInputs=tokenizer.batch_encode_plus(raw_code,add_special_tokens=True,max_length=maxLength,pad_to_max_length=True,return_token_type_ids=True,truncation=True)
#     all_report_input_ids=reportInputs['input_ids']
#     all_report_input_mask=reportInputs['attention_mask']
#     all_code_input_ids=codeInputs['input_ids']
#     all_code_input_mask=codeInputs['attention_mask']
    all_report_input_ids=[[0]for i in range(len(raw_code))]
    all_report_input_mask=[[0]for i in range(len(raw_code))]
    all_code_input_ids=[[0]for i in range(len(raw_code))]
    all_code_input_mask=[[0]for i in range(len(raw_code))]
    all_report_input_ids = torch.tensor(all_report_input_ids, dtype=torch.long)
    all_report_input_mask = torch.tensor(all_report_input_mask, dtype=torch.long)
    all_code_input_ids = torch.tensor(all_code_input_ids, dtype=torch.long)
    all_code_input_mask = torch.tensor(all_code_input_mask, dtype=torch.long)
    
    all_cnn_report_idx=torch.tensor(all_cnn_report_idx, dtype=torch.long)
    all_code_idx=torch.tensor(all_code_idx, dtype=torch.long)
    all_labels = torch.tensor(all_labels, dtype=torch.long)
    pickle.dump([all_report_input_ids, all_report_input_mask, all_code_input_ids, all_code_input_mask,all_cnn_report_idx,
                          all_code_idx, all_labels],open("/data/hdj/cross_project_trans/encode_data/"+project+"parameters_"+ttype+".in", "wb"),protocol = 4)
    dataset = TensorDataset(all_report_input_ids, all_report_input_mask, all_code_input_ids, all_code_input_mask,all_cnn_report_idx,
                          all_code_idx, all_labels)
#     if ttype=='train':
    return dataset
#     else:
#         return dataset,data

In [6]:

# from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, AdamW
project='swt_'
more_train_data=pd.read_pickle('/data/hdj/cross_project_trans/report/bert/'+project+'more_train_data.pkl')
# swt_more_train_data=pd.read_pickle('/home/hdj/cross_project_trans/report/swt_more_train_data.pkl')
# zxing_more_train_data=pd.read_pickle('/home/hdj/cross_project_trans/report/zxing_more_train_data.pkl')
# tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
# project='aspectj_'
# project='swt_'
# project='zxing_'
train_set=data_tensor_generate(more_train_data,project,tokenizer=None,maxLength=256)
batch_size=8
training_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=False)
    #
# raw_report[:3]

In [16]:
batch_size=64
training_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=False)

In [11]:
# #直接从保存的文件里读取train和test数据
batch_size=32
aspectj_train = pickle.load(open('/data/hdj/cross_project_trans/encode_data/'+project+'parameters_train.in','rb'))
all_report_input_ids, all_report_input_mask, all_code_input_ids, all_code_input_mask,all_cnn_report_idx,all_code_idx, all_labels=aspectj_train[0],aspectj_train[1],aspectj_train[2],aspectj_train[3],aspectj_train[4],aspectj_train[5],aspectj_train[6]
train_set = TensorDataset(all_report_input_ids, all_report_input_mask, all_code_input_ids, all_code_input_mask,all_cnn_report_idx,
                          all_code_idx, all_labels)
training_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)
train_set.tensors[0].shape,train_set.tensors[1].shape,train_set.tensors[2].shape,train_set.tensors[3].shape,train_set.tensors[4].shape,train_set.tensors[5].shape,train_set.tensors[6].shape

(torch.Size([21384, 1]),
 torch.Size([21384, 1]),
 torch.Size([21384, 1]),
 torch.Size([21384, 1]),
 torch.Size([21384, 308]),
 torch.Size([21384, 100, 40]),
 torch.Size([21384]))

In [9]:
# loss = nn.MarginRankingLoss()
# input1 = torch.tensor([[1.0,2.0,3.0]], requires_grad=True)
# input2 = torch.tensor([[4.0,5.0,6.0]], requires_grad=True)
# print(input1)
# print(input2)
# # target = torch.randn(3).sign()
# target=torch.tensor([[1]])
# output = loss(input1, input2, target)
# output

In [15]:
len(w[0][0])

300

In [17]:
# t_total = len(train_x1) // args.gradient_accumulation_steps * args.num_train_epochs
model.zero_grad()
model.train()
w= pickle.load(open('/data/hdj/cross_project_trans/report/bert/'+project+'test_middle/w_parameters.in','rb'))
# embedding = nn.Embedding.from_pretrained(torch.tensor(aspectj_w[0]))#从已有的weight导入embedding
# swt_w= pickle.load(open('/home/hdj/cross_project_trans/report/swt_test_middle/w_parameters.in','rb'))
# w= pickle.load(open('/home/hdj/cross_project_trans/report/zxing_test_middle/w_parameters.in','rb'))
embedding = nn.Embedding.from_pretrained(torch.tensor(w[0]))#从已有的weight导入embedding
# scheduler = get_linear_schedule_with_warmup(optimizer, args.warmup_steps, t_total)
# batch_size=128
for epoche in range(3):
    total_acc=0.0
    total_f1=0.0
    total_nums=0
    i=0
    num_batch = len(train_set) / batch_size #总共训练次数
    
    for _, data in enumerate(training_loader, 0):#start=0 默认就是0
        #这是NPCNN需要的数据
        train_x1_batch=embedding(data[5])
        train_x2_batch=embedding(data[4])
        y_train=data[6].to(device, dtype=torch.long).cuda(non_blocking=True)
#         print(train_x1_batch.shape,train_x2_batch.shape)
        train_x1_batch = train_x1_batch.reshape(train_x1_batch.shape[0]*train_x1_batch.shape[1],train_x1_batch.shape[2],train_x1_batch.shape[3])
        train_x1_batch = train_x1_batch.to(device, dtype=torch.float).cuda(non_blocking=True)
        train_x2_batch = train_x2_batch.to(device, dtype=torch.float).cuda(non_blocking=True)
        y_train = y_train.to(device, dtype=torch.long).cuda(non_blocking=True)
#         print('train_x1_batch ',train_x1_batch.shape)
#         print('train_x2_batch ',train_x2_batch.shape)
#         print('y_train ',y_train.shape)
        #NPCNN需要的数据
        #codeBert需要的数据
        report_ids=None
        report_mask=None
        code_ids=None
        code_mask=None
#         report_ids=data[0].to(device, dtype=torch.long).cuda(non_blocking=True)
#         report_mask=data[1].to(device, dtype=torch.long).cuda(non_blocking=True)
#         code_ids=data[2].to(device, dtype=torch.long).cuda(non_blocking=True)
#         code_mask=data[3].to(device, dtype=torch.long).cuda(non_blocking=True)
        #codeBert需要的数据
#         outputs=model(train_x1_batch,train_x2_batch,report_ids,report_mask,code_ids,code_mask)
        outputs=model(train_x1_batch,train_x2_batch,report_ids,report_mask,code_ids,code_mask)
        pred_choice = outputs.max(1)[1]
#         print('cos_simi type',type(cos_simi),cos_simi)
#         print(type(y_train),y_train)
#         print('output y_train shape:',outputs.shape,y_train.shape)
        loss=loss_fun(outputs, y_train)
#         print('codeEmd shape:',codeEmd.shape,reportEmd.shape,y_train.shape)
#         lossRank=lossRankFunction(codeEmd,reportEmd,y_train.reshape(-1,1))
        eval_loss += loss.item()
        
#         if args.gradient_accumulation_steps > 1:
#              loss = loss / args.gradient_accumulation_steps
                
        loss.backward()
#         lossRank.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
#         if(i+1)%args.gradient_accumulation_steps == 0:
        optimizer.step()
#             scheduler.step()
        model.zero_grad()
        correct = pred_choice.eq(y_train).cpu().sum()
        metrics = compute_metrics(pred_choice.cpu().numpy(), y_train.cpu().numpy())
#         print(metrics)
#         if i %4 == 0:
#             print('[',epoche,':',i,'/',num_batch,']',"loss :%.4f" % loss.item(),' , accuracy: ',correct.item() / float(batch_size) ,'true nums:',sum(y_train.cpu().numpy()),' ration :',sum(y_train.cpu().numpy())/len(y_train),"acc :",metrics['acc']," f1 :",metrics['f1'])
        #print(train_x1_batch.shape,train_x2_batch.shape,y_train.shape)
        total_acc+=metrics['acc']
        total_f1+=metrics['f1']
        total_nums+=1
        if i%20==0:
            print('[',epoche,':',i,'/',num_batch,']',"loss :%.4f" % loss.item(),' , accuracy: ',correct.item() / float(batch_size) ,'true nums:',sum(y_train.cpu().numpy()),' ration :',sum(y_train.cpu().numpy())/len(y_train),"acc :",metrics['acc']," f1 :",metrics['f1'])
#         file_path = '/home/hdj/tencent/textCNN/model_v2_age_0606_predicted.hdf5'
        i+=1
    curr_acc=total_acc/total_nums
    curr_f1=total_f1/total_nums
    print(epoche,total_acc,total_nums,curr_acc,curr_f1)

[ 0 : 0 / 668.25 ] loss :0.5663  , accuracy:  0.75 true nums: 5  ration : 0.15625 acc : 0.75  f1 : 0.0
[ 0 : 20 / 668.25 ] loss :0.5152  , accuracy:  0.78125 true nums: 7  ration : 0.21875 acc : 0.78125  f1 : 0.36363636363636365
[ 0 : 40 / 668.25 ] loss :0.5361  , accuracy:  0.65625 true nums: 10  ration : 0.3125 acc : 0.65625  f1 : 0.0
[ 0 : 60 / 668.25 ] loss :0.4456  , accuracy:  0.8125 true nums: 10  ration : 0.3125 acc : 0.8125  f1 : 0.5714285714285715
[ 0 : 80 / 668.25 ] loss :0.4698  , accuracy:  0.78125 true nums: 6  ration : 0.1875 acc : 0.78125  f1 : 0.2222222222222222
[ 0 : 100 / 668.25 ] loss :0.6273  , accuracy:  0.6875 true nums: 12  ration : 0.375 acc : 0.6875  f1 : 0.2857142857142857
[ 0 : 120 / 668.25 ] loss :0.5200  , accuracy:  0.84375 true nums: 4  ration : 0.125 acc : 0.84375  f1 : 0.28571428571428575
[ 0 : 140 / 668.25 ] loss :0.6753  , accuracy:  0.6875 true nums: 11  ration : 0.34375 acc : 0.6875  f1 : 0.28571428571428575
[ 0 : 160 / 668.25 ] loss :0.5703  , acc

/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


[ 1 : 180 / 668.25 ] loss :0.3823  , accuracy:  0.8125 true nums: 6  ration : 0.1875 acc : 0.8125  f1 : 0.25
[ 1 : 200 / 668.25 ] loss :0.4018  , accuracy:  0.8125 true nums: 6  ration : 0.1875 acc : 0.8125  f1 : 0.4
[ 1 : 220 / 668.25 ] loss :0.4476  , accuracy:  0.75 true nums: 10  ration : 0.3125 acc : 0.75  f1 : 0.5
[ 1 : 240 / 668.25 ] loss :0.4923  , accuracy:  0.75 true nums: 11  ration : 0.34375 acc : 0.75  f1 : 0.42857142857142855
[ 1 : 260 / 668.25 ] loss :0.3962  , accuracy:  0.8125 true nums: 9  ration : 0.28125 acc : 0.8125  f1 : 0.5
[ 1 : 280 / 668.25 ] loss :0.5408  , accuracy:  0.71875 true nums: 12  ration : 0.375 acc : 0.71875  f1 : 0.47058823529411764
[ 1 : 300 / 668.25 ] loss :0.5278  , accuracy:  0.78125 true nums: 5  ration : 0.15625 acc : 0.78125  f1 : 0.22222222222222224
[ 1 : 320 / 668.25 ] loss :0.3716  , accuracy:  0.8125 true nums: 9  ration : 0.28125 acc : 0.8125  f1 : 0.5
[ 1 : 340 / 668.25 ] loss :0.4763  , accuracy:  0.75 true nums: 10  ration : 0.3125 a

In [23]:
# #之前版本的代码
# t_total = len(train_x1) // args.gradient_accumulation_steps * args.num_train_epochs
# model.zero_grad()
# model.train()
# # scheduler = get_linear_schedule_with_warmup(optimizer, args.warmup_steps, t_total)
# batch_size=128
# for epoche in range(10):
#     total_acc=0.0
#     total_nums=0
#     i=0
#     num_batch = len(train_x1) / batch_size #总共训练次数
#     for train_x1_batch,train_x2_batch,y_train in  minibatches(train_x1,train_x2,train_y,batch_size,True):
#         train_x1_batch = train_x1_batch.reshape(train_x1_batch.shape[0]*train_x1_batch.shape[1],train_x1_batch.shape[2],train_x1_batch.shape[3])
#         train_x1_batch = torch.tensor(train_x1_batch).to(device, dtype=torch.float).cuda(non_blocking=True)
#         train_x2_batch = torch.tensor(train_x2_batch).to(device, dtype=torch.float).cuda(non_blocking=True)
#         y_train = torch.tensor(y_train).to(device, dtype=torch.long).cuda(non_blocking=True)
#         outputs=model(train_x1_batch,train_x2_batch)
#         pred_choice = outputs.max(1)[1]
        
# #         print(type(y_train),y_train)
#         loss=loss_fun(outputs, y_train)
#         eval_loss += loss.item()
        
# #         if args.gradient_accumulation_steps > 1:
# #              loss = loss / args.gradient_accumulation_steps
                
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
# #         if(i+1)%args.gradient_accumulation_steps == 0:
#         optimizer.step()
# #             scheduler.step()
#         model.zero_grad()
#         correct = pred_choice.eq(y_train).cpu().sum()
#         metrics = compute_metrics(pred_choice.cpu().numpy(), y_train.cpu().numpy())
# #         print(metrics)
#         if i %4 == 0:
#             print('[',epoche,':',i,'/',num_batch,']',"loss :%.4f" % loss.item(),' , accuracy: ',correct.item() / float(batch_size) ,'true nums:',sum(y_train.cpu().numpy()),' ration :',sum(y_train.cpu().numpy())/len(y_train),"acc :",metrics['acc']," f1 :",metrics['f1'])
#         #print(train_x1_batch.shape,train_x2_batch.shape,y_train.shape)
# #         file_path = '/home/hdj/tencent/textCNN/model_v2_age_0606_predicted.hdf5'
#         i+=1
    

# 测试部分

In [26]:
torch.save(model, '/data/hdj/cross_project_trans/report/bert/npcnn_model_0621.pkl')

In [8]:
model=torch.load('/home/hdj/NPCNN_hdj/model.pkl')

In [5]:
X = pickle.load(open('/home/hdj/NPCNN_hdj/parameters.in','rb'))

train_report,train_code,train_labels,W = X[0],X[1],X[2],X[3]
embedding = nn.Embedding.from_pretrained(torch.tensor(W))#从已有的weight导入embedding

In [11]:
aspectj_w= pickle.load(open('/home/hdj/cross_project_trans/report/aspectj_test_middle/w_parameters.in','rb'))
embedding = nn.Embedding.from_pretrained(torch.tensor(aspectj_w[0]))#从已有的weight导入embedding

In [13]:
#将test数据集合并
report_path='/data/hdj/cross_project_trans/report/bert/'
all_test=pd.read_pickle('/data/hdj/cross_project_trans/report/bert/'+project+'test_middle/test'+str(0)+'.pkl')
# all_test=pd.read_pickle('/home/hdj/cross_project_trans/report/swt_test_middle/test'+str(0)+'.pkl')
# all_test=pd.read_pickle('/home/hdj/cross_project_trans/report/zxing_test_middle/test'+str(0)+'.pkl')
test_path=report_path+project+'test/'
test_num=len(os.listdir(test_path))
for test_index in range(1,100):
# for test_index in range(1,20):
# for test_index in range(1,4):
    test=pd.read_pickle('/data/hdj/cross_project_trans/report/bert/'+project+'test_middle/test'+str(test_index)+'.pkl')
#     test=pd.read_pickle('/home/hdj/cross_project_trans/report/swt_test_middle/test'+str(test_index)+'.pkl')
#     test=pd.read_pickle('/home/hdj/cross_project_trans/report/zxing_test_middle/test'+str(test_index)+'.pkl')
    all_test=pd.concat([all_test,test])
test_set=data_tensor_generate(all_test,project,tokenizer=None,maxLength=256,ttype='test')
print(test_set.tensors[0].shape,test_set.tensors[1].shape,test_set.tensors[2].shape,test_set.tensors[3].shape,test_set.tensors[4].shape,test_set.tensors[5].shape,test_set.tensors[6].shape)

torch.Size([30000, 1]) torch.Size([30000, 1]) torch.Size([30000, 1]) torch.Size([30000, 1]) torch.Size([30000, 308]) torch.Size([30000, 100, 40]) torch.Size([30000])


In [10]:
# test_set.tensors[0][1]

In [18]:
top10_all = []
MRR_all = []
MAP_all = []
#直接从保存的文件里读取train和test数据
# batch_size=4
test = pickle.load(open('/data/hdj/cross_project_trans/encode_data/'+project+'parameters_test.in','rb'))
# test = pickle.load(open('/data/hdj/cross_project_trans/encode_data/swt_parameters_test.in','rb'))
# test = pickle.load(open('/data/hdj/cross_project_trans/encode_data/zxing_parameters_test.in','rb'))
all_report_input_ids, all_report_input_mask, all_code_input_ids, all_code_input_mask,all_cnn_report_idx,all_code_idx, all_labels=test[0],test[1],test[2],test[3],test[4],test[5],test[6]
test_set = TensorDataset(all_report_input_ids, all_report_input_mask, all_code_input_ids, all_code_input_mask,all_cnn_report_idx,
                          all_code_idx, all_labels)

# test_set=data_tensor_generate(all_test,tokenizer=tokenizer,maxLength=10)
test_loader = DataLoader(dataset=test_set, batch_size=32, shuffle=False, num_workers=4, drop_last=False)
test_p=[]
test_y_labels=[]
for i, data in enumerate(test_loader, 0):#start=0 默认就是0
    with torch.no_grad():
        train_x1_batch=embedding(data[5])
        train_x2_batch=embedding(data[4])
        test_labels=data[6]
#         print()
        if i%20==0:
            print(i,'/',len(test_set)/batch_size,train_x1_batch.shape,train_x2_batch.shape)
        #NPCNN的数据处理
        train_x1_batch = train_x1_batch.reshape(train_x1_batch.shape[0]*train_x1_batch.shape[1],train_x1_batch.shape[2],train_x1_batch.shape[3])
        train_x1_batch = train_x1_batch.to(device, dtype=torch.float).cuda(non_blocking=True)
        train_x2_batch = train_x2_batch.to(device, dtype=torch.float).cuda(non_blocking=True)
        #NPCNN的数据处理
        
        #codeBert的数据处理
        
        report_ids= None #data[0].to(device, dtype=torch.long).cuda(non_blocking=True)
        report_mask=None #data[1].to(device, dtype=torch.long).cuda(non_blocking=True)
        code_ids=   None #data[2].to(device, dtype=torch.long).cuda(non_blocking=True)
        code_mask=  None #data[3].to(device, dtype=torch.long).cuda(non_blocking=True)
        #codeBert需要的数据
        outputs=model(train_x1_batch,train_x2_batch,report_ids,report_mask,code_ids,code_mask)
        y_train = test_labels.to(device, dtype=torch.long).cuda(non_blocking=True)
        loss=loss_fun(outputs, y_train)
        if i%20==0:
            print('test loss :',loss)
#         outputs=model(train_x1_batch,train_x2_batch)
        outputs = F.softmax(torch.tensor(outputs))
        test_p.extend(outputs[:,1].cpu())
        test_y_labels.extend(test_labels)


0 / 937.5 torch.Size([32, 100, 40, 300]) torch.Size([32, 308, 300])
test loss : tensor(0.6454, device='cuda:0')


/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


20 / 937.5 torch.Size([32, 100, 40, 300]) torch.Size([32, 308, 300])
test loss : tensor(0.7123, device='cuda:0')
40 / 937.5 torch.Size([32, 100, 40, 300]) torch.Size([32, 308, 300])
test loss : tensor(0.5098, device='cuda:0')
60 / 937.5 torch.Size([32, 100, 40, 300]) torch.Size([32, 308, 300])
test loss : tensor(0.5219, device='cuda:0')
80 / 937.5 torch.Size([32, 100, 40, 300]) torch.Size([32, 308, 300])
test loss : tensor(0.6948, device='cuda:0')
100 / 937.5 torch.Size([32, 100, 40, 300]) torch.Size([32, 308, 300])
test loss : tensor(0.7864, device='cuda:0')
120 / 937.5 torch.Size([32, 100, 40, 300]) torch.Size([32, 308, 300])
test loss : tensor(0.5468, device='cuda:0')
140 / 937.5 torch.Size([32, 100, 40, 300]) torch.Size([32, 308, 300])
test loss : tensor(0.7559, device='cuda:0')
160 / 937.5 torch.Size([32, 100, 40, 300]) torch.Size([32, 308, 300])
test loss : tensor(0.6512, device='cuda:0')
180 / 937.5 torch.Size([32, 100, 40, 300]) torch.Size([32, 308, 300])
test loss : tensor(0.7

In [13]:
outputs.shape

torch.Size([16, 2])

In [20]:
len(test_p)
for i,(p,y) in enumerate(zip(test_p,test_y_labels)):
    print(i,p,y)
    if i==300:
        break

0 tensor(0.5151) tensor(0)
1 tensor(0.3715) tensor(0)
2 tensor(0.5810) tensor(0)
3 tensor(0.2859) tensor(1)
4 tensor(0.5733) tensor(0)
5 tensor(0.5540) tensor(0)
6 tensor(0.4765) tensor(0)
7 tensor(0.5426) tensor(0)
8 tensor(0.2721) tensor(0)
9 tensor(0.5004) tensor(0)
10 tensor(0.5678) tensor(0)
11 tensor(0.2542) tensor(0)
12 tensor(0.2076) tensor(0)
13 tensor(0.1398) tensor(0)
14 tensor(0.1988) tensor(0)
15 tensor(0.1193) tensor(0)
16 tensor(0.2959) tensor(0)
17 tensor(0.1648) tensor(0)
18 tensor(0.4333) tensor(1)
19 tensor(0.1595) tensor(0)
20 tensor(0.2943) tensor(0)
21 tensor(0.1101) tensor(0)
22 tensor(0.6559) tensor(0)
23 tensor(0.1448) tensor(0)
24 tensor(0.5029) tensor(0)
25 tensor(0.2134) tensor(0)
26 tensor(0.0268) tensor(0)
27 tensor(0.5396) tensor(0)
28 tensor(0.1535) tensor(0)
29 tensor(0.2034) tensor(0)
30 tensor(0.5325) tensor(0)
31 tensor(0.0716) tensor(0)
32 tensor(0.2905) tensor(0)
33 tensor(0.3880) tensor(0)
34 tensor(0.2277) tensor(0)
35 tensor(0.7974) tensor(0)
36

In [21]:
print(len(test_p),len(test_y_labels))
for i in range(100):
# for i in range(20):
# for i in range(4):
#     top10,MRR,MAP = eval_y(test_p[i*1000:(i+1)*1000], test_y_labels[i*1000:(i+1)*1000])
    top10,MRR,MAP = eval_y(test_p[i*300:(i+1)*300], test_y_labels[i*300:(i+1)*300])
    top10_all.append(top10)
    MRR_all.append(MRR)
    MAP_all.append(MAP)

print('top10:', np.mean(top10_all))
print('MRR:', np.mean(MRR_all))
print('MAP:',np.mean(MAP_all))
#16:17

30000 30000
出现pos_num为0的情况 0
出现pos_num为0的情况 0
出现pos_num为0的情况 0
出现pos_num为0的情况 0
出现pos_num为0的情况 0
出现pos_num为0的情况 0
出现pos_num为0的情况 0
top10: 0.225
MRR: 0.1850253038637951
MAP: 0.1161799111839623


In [5]:
# MAP
print((0.31+0.23+0.14+0.25+0.43+0.22)/6)
print((0.44+0.39+0.16+0.40+0.49+0.37)/6)
print((0.44+0.37+0.18+0.41+0.50+0.43)/6)
print((0.41+0.34+0.20+0.37+0.52+0.32)/6)
print((0.46+0.39+0.20+0.43+0.54+0.45)/6)


0.26333333333333336
0.375
0.38833333333333336
0.35999999999999993
0.4116666666666667


In [6]:
#MRR
print((0.37+0.30+0.18+0.28+0.48+0.32)/6)
print((0.51+0.47+0.21+0.46+0.55+0.44)/6)
print((0.51+0.46+0.24+0.47+0.57+0.51)/6)
print((0.51+0.45+0.28+0.45+0.60+0.52)/6)
print((0.53+0.48+0.26+0.50+0.60+0.55)/6)


0.32166666666666666
0.44
0.45999999999999996
0.4683333333333333
0.48666666666666664


In [7]:
# TOP10
print((0.60+0.51+0.32+0.51+0.71+0.57)/6)
print((0.75+0.74+0.21+0.46+0.55+0.44)/6)
print((0.73+0.74+0.38+0.75+0.78+0.68)/6)
print((0.78+0.74+0.51+0.80+0.80+0.85)/6)
print((0.75+0.72+0.45+0.77+0.81+0.70)/6)

0.5366666666666666
0.525
0.6766666666666666
0.7466666666666666
0.7000000000000001


In [31]:
# MAP
(0.43+ 0.36 + 0.19 + 0.40 +0.52 + 0.42)/6
# MRR
# (0.50 + 0.45 + 0.25 + 0.45 + 0.59 +0.53)/6
#TOP 10
# (0.73 + 0.72 + 0.40 +0.73 + 0.80 + 0.69)/6

0.38666666666666666

In [9]:
MAP
fl=0.41
for num in [0.26,0.38,0.39,0.35]:
    print((fl-num)/num)

0.5769230769230768
0.07894736842105256
0.05128205128205118
0.17142857142857143


In [10]:
# MRR
fl=0.49
for num in [0.32,0.44,0.46,0.47]:
    print((fl-num)/num)

0.5312499999999999
0.1136363636363636
0.06521739130434775
0.04255319148936174


In [11]:
fl=0.70
for num in [0.54,0.53,0.68]:
    print((fl-num)/num)

0.2962962962962961
0.32075471698113195
0.029411764705882214


In [13]:
#ablation
org=[0.708,0.682,0.397,0.739,0.751,0.67]
fine=[0.746,0.714,0.449,0.771,0.810, 0.69]
for o,f in zip(org,fine):
    print((f-o)/o)

0.053672316384180845
0.046920821114369377
0.13098236775818636
0.043301759133964855
0.07856191744340886
0.029850746268656577


In [14]:
#ablation MRR
org=[0.503,0.453,0.230,0.464,0.568,0.51]
fine=[0.533,0.478,0.260,0.497, 0.596 ,0.548]
for o,f in zip(org,fine):
    print((f-o)/o)

0.059642147117296276
0.05518763796909485
0.13043478260869565
0.07112068965517235
0.04929577464788737
0.0745098039215687


In [15]:
#ablation MAP
org=[0.429,0.370,0.178,0.401,0.503,0.44]
fine=[0.457,0.392,0.204,0.431,0.535 ,0.45]
for o,f in zip(org,fine):
    print((f-o)/o)

0.06526806526806533
0.05945945945945951
0.14606741573033705
0.0748129675810473
0.06361829025844937
0.02272727272727275


In [4]:
#被废弃代码
#import pickle
# index=2
# print("loading data for training")
# test_num = 58
# X = pickle.load(open('/home/hdj/NPCNN_hdj/parameters.in','rb'))

# train_report,train_code,train_labels,W = X[0],X[1],X[2],X[3]
# embedding = nn.Embedding.from_pretrained(torch.tensor(W))#从已有的weight导入embedding
# train_code = embedding(torch.tensor(train_code))
# train_report =embedding(torch.tensor(train_report))
# # train_report = np.array(train_report)
# # train_code = np.array(train_code)
# train_labels = np.array(train_labels)
# # W = np.array(W)
# # print(np.array(train_report).shape, np.array(train_code).shape, np.array(train_labels).shape, np.array(W).shape)
# print("finish reading")
# #在W中找到所需的单词向量，维度多一维
# # train_code = W[train_code]
# # train_report = W[train_report]
# # print(np.array(train_code).shape,np.array(train_report).shape)

# train_x1 = train_code
# train_x2 = train_report
# train_y = train_labels


loading data for training
finish reading


# 为python脚本NP_CNN_hdj准备数据

In [4]:
#需要为数据的code生成文本形式
def get_testsets(test_code,test_report,test_label):
    code_x = test_code[0]
    for i in range(300-1):
        code_x = np.vstack((code_x,test_code[i+1]))
    report_x = test_report
    y = test_label
    return code_x,report_x,y
def code_merge(raw_code_list):
    code_list=[]
    for raw_code in raw_code_list:
        code_list.append(raw_code['text'])
    code_text=' '.join(code_list)
    return code_text

def data_tensor_generate_test(data,project,tokenizer,maxLength=512,ttype='train'):
    '''
        输入是dataframe 包含bug_id	cnn_report_idx	code_idx	label	path	raw_report	raw_code
    '''
    #将raw_code里dict形式合并生成code_text
    data['raw_code_text']=data['raw_code'].apply(code_merge)
    #生成完成
    all_labels=list(data['label'])
    all_cnn_report_idx=list(data['cnn_report_idx'])
    all_code_idx=list(data['code_idx'])
    raw_report=list(data['raw_report'])
    raw_code=list(data['raw_code_text'])
#     reportInputs=tokenizer.batch_encode_plus(raw_report,add_special_tokens=True,max_length=maxLength,pad_to_max_length=True,return_token_type_ids=True,truncation=True)
#     codeInputs=tokenizer.batch_encode_plus(raw_code,add_special_tokens=True,max_length=maxLength,pad_to_max_length=True,return_token_type_ids=True,truncation=True)
#     all_report_input_ids=reportInputs['input_ids']
#     all_report_input_mask=reportInputs['attention_mask']
#     all_code_input_ids=codeInputs['input_ids']
#     all_code_input_mask=codeInputs['attention_mask']
    all_report_input_ids=[[0]for i in range(len(raw_code))]
    all_report_input_mask=[[0]for i in range(len(raw_code))]
    all_code_input_ids=[[0]for i in range(len(raw_code))]
    all_code_input_mask=[[0]for i in range(len(raw_code))]
    all_report_input_ids = torch.tensor(all_report_input_ids, dtype=torch.long)
    all_report_input_mask = torch.tensor(all_report_input_mask, dtype=torch.long)
    all_code_input_ids = torch.tensor(all_code_input_ids, dtype=torch.long)
    all_code_input_mask = torch.tensor(all_code_input_mask, dtype=torch.long)
    
    all_cnn_report_idx=torch.tensor(all_cnn_report_idx, dtype=torch.long)
    all_code_idx=torch.tensor(all_code_idx, dtype=torch.long)
    all_labels = torch.tensor(all_labels, dtype=torch.long)
    pickle.dump([all_report_input_ids, all_report_input_mask, all_code_input_ids, all_code_input_mask,all_cnn_report_idx,
                          all_code_idx, all_labels],open("/data/hdj/cross_project_trans/report/bert/"+project+"test_middle/test_"+ttype+".in", "wb"),protocol = 4)
    dataset = TensorDataset(all_report_input_ids, all_report_input_mask, all_code_input_ids, all_code_input_mask,all_cnn_report_idx,
                          all_code_idx, all_labels)
#     if ttype=='train':
    return dataset
#     else:
#         return dataset,data
import torch
project='swt_'
for test_index in range(100):
    test=pd.read_pickle('/data/hdj/cross_project_trans/report/bert/'+project+'test_middle/test'+str(test_index)+'.pkl')
    data_tensor_generate_test(test,project,tokenizer=None,maxLength=256,ttype=str(test_index))

# 为python脚本NP_CNN_hdj准备数据结束

In [18]:
X = pickle.load(open("/data/hdj/cross_project_trans/report/bert/test_middle0.csv","rb"))
import pickle
Y = pickle.load(open("/data/hdj/cross_project_trans/report/bert/swt_test_middle/test0.pkl","rb"))
# #老版本代码
# top10_all = []
# MRR_all = []
# MAP_all = []

# for test_index in range(58):
#     X = pickle.load(open("/home/hdj/NPCNN_hdj/middle/test_middle"+str(test_index)+".csv","rb"))
#     test_report,test_code,test_labels = X[0],X[1],X[2]
#     test_report = np.array(test_report,dtype='int')[:,:308]#TODO 308 表示最大report长度+2*filter_h
#     test_code = np.array(test_code,dtype='int')
#     test_labels = np.array(test_labels,dtype='int')
#    # print(test_code.shape,test_report.shape,train_labels.shape)
#     test_code_x, test_report_x, test_label = get_testsets(test_code,test_report,test_labels)
# #     test_code_x = W[test_code_x].reshape(300*config.codeLineNums,config.wordNums,config.embedding_size)#300*50 TODO 100 表示最大语句数量 40表示每个statement最大词汇
# #     test_report_x = W[test_report_x]
#     test_code_x = embedding(torch.tensor(test_code_x)).reshape(300*config.codeLineNums,config.wordNums,config.embedding_size)
#     test_report_x=embedding(torch.tensor(test_report_x))
#     test_code_x = torch.tensor(test_code_x).to(device, dtype=torch.float).cuda(non_blocking=True)
#     test_report_x = torch.tensor(test_report_x).to(device, dtype=torch.float).cuda(non_blocking=True)
#     print('test_code_x :',test_code_x.shape,'test_report_x :',test_report_x.shape)
#     output=model(test_code_x,test_report_x)
# #     y_train = torch.tensor(y_train).to(device, dtype=torch.long).cuda(non_blocking=True)
#     #print(test_code_x.shape,test_report_x.shape)
    
#     #pre,label = sess.run([prediction,output_label],feed_dict={x1:test_code_x,x2:test_report_x,keep_prob:0.25})
#     #print(pre,label)
#     test_p = output[:,1]
#     test_y_labels = test_labels
#    # top10 = eval_y(test_p, label, test_y_labels)
#     #top10_all.append(top10)
# #print('top10:', np.mean(top10_all))


#     top10,MRR,MAP = eval_y(test_p, test_y_labels)
#     top10_all.append(top10)
#     MRR_all.append(MRR)
#     MAP_all.append(MAP)

# print('top10:', np.mean(top10_all))
# print('MRR:', np.mean(MRR_all))
# print('MAP:',np.mean(MAP_all))
# #16:17

In [7]:
# codeInput = torch.randn(32*100, 40,300)
# reportInput=torch.randn(32,150,300)
# output=cnnMode(codeInput,reportInput)
# print(output.shape)